In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('cgi_biomarkers.tsv',sep='\t')
df = df[df['Alteration type']=='MUT']
df = df[df['Association']=='Responsive']
selection = ['CM','LUAD','LUSC','NSCLC']
df = df[df['Primary Tumor type'].isin(selection)]
df = df[~df['Biomarker'].str.contains('oncogenic mutation|proximal exon|mutation in exon|inframe insertion')]
df = df[['Biomarker','Gene','Alteration','Drug','Primary Tumor type','Drug full name','Drug status']].drop_duplicates(['Biomarker','Primary Tumor type'],keep='first')
df['Alteration'] = df['Biomarker'].str.split('(',expand=True)[1].str.split(')',expand=True)[0]
df['Biomarker'] = df['Biomarker'].str.split(' ',expand=True)[0]
df = df[~df['Alteration'].isnull()]
df = df[df['Drug']!='[]']
df_list = df.values.tolist()
final_list = []
for row in df_list:
    if ',' in row[2]:
        alterations = row[2].split(',')
        num_alt = len(alterations)
        for i in range(0,num_alt):
            new_row = [row[0] , row[1] , alterations[i] , row[3] , row[4] , row[5]]
            final_list.append(new_row)
    else:
        final_list.append(row)
final_df = pd.DataFrame(final_list,columns=df.columns)
final_df['Biomarker'] = final_df['Gene'] +'_'+ final_df['Alteration']

def inh_type(x):
    if '(' in x:
        if 'etc' in x:
            y = x.split(' (')[0]
        else:
            y = x.split('(')[1].split(')')[0]
    else:
        y = x
    return y

def drug_name(x):
    if '[' in x:
        y = x.split('[')[1].split(']')[0]
    else:
        y = x
    return y

final_df['Inhibitor type'] = final_df['Drug full name'].apply(lambda x: inh_type(x))
final_df['Drug'] = final_df['Drug'].apply(lambda x: drug_name(x))
final_df['Approved'] = final_df['Drug status'].replace({None:False,'Approved':True, 'FDA approved':True})
final_df.drop(columns=['Drug full name','Drug status'],inplace=True)
# final_df.drop_duplicates(['Biomarker','Primary Tumor type'],keep = 'first',inplace=True)
final_df

,Biomarker,Gene,Alteration,Drug,Primary Tumor type,Inhibitor type,Approved
0,ALK_E1408V,ALK,E1408V,Brigatinib,LUAD,Pan-TK inhibitor,False
1,ALK_L1196M,ALK,L1196M,Brigatinib,LUAD,Pan-TK inhibitor,False
2,ALK_L1196M,ALK,L1196M,Ceritinib,LUAD,ALK inhibitor,False
3,ALK_S1206Y,ALK,S1206Y,Ceritinib,LUAD,ALK inhibitor,False
4,ALK_G1269A,ALK,G1269A,Ceritinib,LUAD,ALK inhibitor,False
...,...,...,...,...,...,...,...
77,PDGFRA_H845Y,PDGFRA,H845Y,Imatinib,CM,BCR-ABL inhibitor 1st gen&KIT inhibitor,False
78,PDGFRA_G853D,PDGFRA,G853D,Imatinib,CM,BCR-ABL inhibitor 1st gen&KIT inhibitor,False
79,ROS1_G2032R,ROS1,G2032R,Cabozantinib,LUAD,Pan-kinase inhibitor,False
80,ROS1_S1986Y,ROS1,S1986Y,Lorlatinib,LUAD,ALK&ROS1 inhibitor,False


In [4]:
final_df.groupby('Approved').count()

,Biomarker,Gene,Alteration,Drug,Primary Tumor type,Inhibitor type
Approved,,,,,,
False,80,80,80,80,80,80
True,2,2,2,2,2,2


In [68]:
final_df[final_df['Approved']==True]

,Biomarker,Gene,Alteration,Drug,Primary Tumor type,Inhibitor type,Approved
8,ARAF_S214C,ARAF,S214C,Sorafenib,LUAD,Pan-TK inhibitor,True
54,EGFR_L858R,EGFR,L858R,Erlotinib,NSCLC,EGFR inhibitor 1st gen,True


In [53]:
df1 = final_df.groupby(['Biomarker','Primary Tumor type'],as_index=False).count()
print(len(df1))
df2 = df1[df1['Gene']>1]
multiple = df2['Biomarker'].tolist()

60


In [54]:
cm_df = final_df[final_df['Primary Tumor type']=='CM']
len(cm_df.groupby('Biomarker').count())

25

In [55]:
cm_df

,Biomarker,Gene,Alteration,Drug,Primary Tumor type,Inhibitor type,Approved
9,BRAF_K601R,BRAF,K601R,Trametinib,CM,MEK inhibitors,False
10,BRAF_L597R,BRAF,L597R,Trametinib,CM,MEK inhibitors,False
11,BRAF_V600R,BRAF,V600R,Trametinib,CM,MEK inhibitors,False
13,BRAF_V600R,BRAF,V600R,Dabrafenib,CM,BRAF inhibitor,False
14,BRAF_V600E,BRAF,V600E,Dabrafenib;Trametinib,CM,BRAF inhibitor + MEK inhibitor,False
15,BRAF_V600K,BRAF,V600K,Dabrafenib;Trametinib,CM,BRAF inhibitor + MEK inhibitor,False
18,BRAF_D594G,BRAF,D594G,Sorafenib,CM,Pan-TK inhibitor,False
19,BRAF_G469E,BRAF,G469E,Sorafenib,CM,Pan-TK inhibitor,False
20,BRAF_V600D,BRAF,V600D,Vemurafenib,CM,BRAF inhibitor,False
21,BRAF_V600K,BRAF,V600K,Vemurafenib,CM,BRAF inhibitor,False


In [56]:
luad_df = final_df[final_df['Primary Tumor type']=='LUAD']
len(luad_df.groupby('Biomarker').count())

14

In [57]:
luad_df

,Biomarker,Gene,Alteration,Drug,Primary Tumor type,Inhibitor type,Approved
0,ALK_E1408V,ALK,E1408V,Brigatinib,LUAD,Pan-TK inhibitor,False
1,ALK_L1196M,ALK,L1196M,Brigatinib,LUAD,Pan-TK inhibitor,False
2,ALK_L1196M,ALK,L1196M,Ceritinib,LUAD,ALK inhibitor,False
3,ALK_S1206Y,ALK,S1206Y,Ceritinib,LUAD,ALK inhibitor,False
4,ALK_G1269A,ALK,G1269A,Ceritinib,LUAD,ALK inhibitor,False
5,ALK_I1171T,ALK,I1171T,Ceritinib,LUAD,ALK inhibitor,False
6,ALK_L1198F,ALK,L1198F,Crizotinib,LUAD,ALK inhibitor,False
7,ALK_C1156Y,ALK,C1156Y,Lorlatinib,LUAD,ALK&ROS1 inhibitor,False
8,ARAF_S214C,ARAF,S214C,Sorafenib,LUAD,Pan-TK inhibitor,True
16,BRAF_G466V,BRAF,G466V,Dasatinib,LUAD,BCR-ABL inhibitor 2nd gen,False


In [58]:
lusc_df = final_df[final_df['Primary Tumor type']=='LUSC']
len(lusc_df.groupby('Biomarker').count())

7

In [59]:
lusc_df

,Biomarker,Gene,Alteration,Drug,Primary Tumor type,Inhibitor type,Approved
25,DDR2_I638F,DDR2,I638F,Dasatinib,LUSC,BCR-ABL inhibitor 2nd gen,False
26,DDR2_L239R,DDR2,L239R,Dasatinib,LUSC,BCR-ABL inhibitor 2nd gen,False
27,DDR2_G253C,DDR2,G253C,Dasatinib,LUSC,BCR-ABL inhibitor 2nd gen,False
28,DDR2_G774V,DDR2,G774V,Dasatinib,LUSC,BCR-ABL inhibitor 2nd gen,False
29,DDR2_L63V,DDR2,L63V,Dasatinib,LUSC,BCR-ABL inhibitor 2nd gen,False
30,DDR2_G505S,DDR2,G505S,Dasatinib,LUSC,BCR-ABL inhibitor 2nd gen,False
31,DDR2_S768R,DDR2,S768R,Dasatinib,LUSC,BCR-ABL inhibitor 2nd gen,False


In [60]:
nsclc_df = final_df[final_df['Primary Tumor type']=='NSCLC']
len(nsclc_df.groupby('Biomarker').count())

14

In [61]:
nsclc_df

,Biomarker,Gene,Alteration,Drug,Primary Tumor type,Inhibitor type,Approved
12,BRAF_V600E,BRAF,V600E,Dabrafenib,NSCLC,BRAF inhibitor,False
32,EGFR_T790M,EGFR,T790M,"Rociletinib,HM61713",NSCLC,EGFR inhibitor 3rd gens,False
33,EGFR_L858R,EGFR,L858R,Afatinib,NSCLC,ERBB2 inhibitor&EGFR inhibitor 2nd gen,False
34,EGFR_L861Q,EGFR,L861Q,Afatinib,NSCLC,ERBB2 inhibitor&EGFR inhibitor 2nd gen,False
35,EGFR_G719A,EGFR,G719A,Afatinib,NSCLC,ERBB2 inhibitor&EGFR inhibitor 2nd gen,False
36,EGFR_G719S,EGFR,G719S,Afatinib,NSCLC,ERBB2 inhibitor&EGFR inhibitor 2nd gen,False
37,EGFR_G719C,EGFR,G719C,Afatinib,NSCLC,ERBB2 inhibitor&EGFR inhibitor 2nd gen,False
38,EGFR_G719D,EGFR,G719D,Afatinib,NSCLC,ERBB2 inhibitor&EGFR inhibitor 2nd gen,False
39,EGFR_L747S,EGFR,L747S,Afatinib,NSCLC,ERBB2 inhibitor&EGFR inhibitor 2nd gen,False
40,EGFR_S768I,EGFR,S768I,Afatinib,NSCLC,ERBB2 inhibitor&EGFR inhibitor 2nd gen,False


In [5]:
final_df.to_csv('selected_biomarkers.tsv',sep='\t',index=False)

In [6]:
df = pd.read_csv('selected_biomarkers.tsv',sep='\t')
df

,Biomarker,Gene,Alteration,Drug,Primary Tumor type,Inhibitor type,Approved
0,ALK_E1408V,ALK,E1408V,Brigatinib,LUAD,Pan-TK inhibitor,False
1,ALK_L1196M,ALK,L1196M,Brigatinib,LUAD,Pan-TK inhibitor,False
2,ALK_L1196M,ALK,L1196M,Ceritinib,LUAD,ALK inhibitor,False
3,ALK_S1206Y,ALK,S1206Y,Ceritinib,LUAD,ALK inhibitor,False
4,ALK_G1269A,ALK,G1269A,Ceritinib,LUAD,ALK inhibitor,False
...,...,...,...,...,...,...,...
77,PDGFRA_H845Y,PDGFRA,H845Y,Imatinib,CM,BCR-ABL inhibitor 1st gen&KIT inhibitor,False
78,PDGFRA_G853D,PDGFRA,G853D,Imatinib,CM,BCR-ABL inhibitor 1st gen&KIT inhibitor,False
79,ROS1_G2032R,ROS1,G2032R,Cabozantinib,LUAD,Pan-kinase inhibitor,False
80,ROS1_S1986Y,ROS1,S1986Y,Lorlatinib,LUAD,ALK&ROS1 inhibitor,False
